# Module 3: Principal componentent analysis

In this lab you will learn about **Principal componentent analysis** (PCA).
It is a linear transformation that transforms data points into feature space representation,
which has the following properties:

+ Exactly the **same** dimension as the original data space
+ Reconstruction of data points is optimal in the mean-square-error sense.
+ Principal components are sorted by "effectiveness" at capturing variance.
+ Dimensionality reduction can be done simply by truncating (down selecting the number of) principal components.

For this session, we are going to use **red wine quality** dataset for principal componentent analysis 
and transform its data space into feature space with top 5 principal components.

sklearn API reference:

+ [sklearn.decomposition.PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)


**Note:** Previously you saw PCA in the 8610 (Stat/Math) course.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

## Load dataset

In [ ]:
# Dataset location
DATASET = '/dsa/data/all_datasets/wine-quality/winequality-red.csv'
assert os.path.exists(DATASET)

# Load and shuffle
dataset = pd.read_csv(DATASET, sep=';').sample(frac = 1).reset_index(drop=True)



In [ ]:
X = np.array(dataset.iloc[:, :-1])

X_centered = scale(X, with_std = False)

y = np.array(dataset.quality)

print(dataset.shape)



## PCA with sklearn

The following cell performs principal component analysis and transforms data space into feature space,
which is done by a matrix transform followed by truncation of low variance principal components.
Then you are free to use the obtained features **X_features** as input to your model. 

In [ ]:
pca = PCA(n_components=5)
pca.fit(X_centered)
print(pca.explained_variance_ratio_)
X_features = pca.transform(X_centered)
print('Features shape', X_features.shape)

## Reconstruction of data space

An approximation of original dataset can be recovered by linearly combining high variance principal components.
This demonstrates the connection between data space and feature space.

$$ \hat X = AQ + \mu \approx X $$

**Note**: The difference between **X_synthesized** and **X_reconstructed** is that the former is centered
at the origin because PCA assumes mathematical expectation of input to be zero and input needs centered otherwise.
Therefore, in order to reconstruct the original dataset, the approximation obtained using PCA has to be
shifted back.

In [ ]:
print('Principal components shape', pca.components_.shape)
X_synthesized = np.dot(X_features, pca.components_)
X_reconstructed = X_synthesized + np.mean(X, axis = 0)[np.newaxis, ...]
print('Reconstructed dataset shape', X_reconstructed.shape)

The following cell shows a reconstructed dataset which is an approximation of original dataset.

In [ ]:
pd.DataFrame(X_reconstructed, columns = dataset.columns[:-1]).head()

The following cell shows the original dataset for comparison, and that we would not
lose a whole lot of information by throwing away those low variance principal components.

In [ ]:
dataset.iloc[:5, :-1]

## Approximation error of principal components

As a result, we were able to represent original dataset using principal components
without losing too much information. 
Space consumption was essentially halved.
And these principal components can be used as features for subsequent machine learning pipeline.
The following calculates mean squared error between original dataset and reconstructed dataset for all cells.

In [ ]:
error = X - X_reconstructed
np.mean((error**2))

## Verify eigenstructure of PCA

The solution of PCA could be found by performing **eigen-decomposition** of the covariance matrix of **X**.
This section quickly verifies some of its properties pertaining to eigenstructure.

Some other interesting facts about PCA include that the error and approximation are orthogonal.
Because the approximation consists of high variance principal components while the error solely 
consists of low variance principal components, and all principal components are orthogonal pairwise.

$$ E^T (\hat X- \mu) = 0$$

Moreover, this orthogonality is a desirable property because it is tantamount to achieving minimum mean square error. 
See [Principle of Orthogonality](https://en.wikipedia.org/wiki/Orthogonality_principle).

In [ ]:
np.allclose(np.dot(error.T, X_synthesized), np.zeros((X.shape[1],X.shape[1])))

In [ ]:
np.isclose(np.dot(error.T, X_synthesized), np.zeros((X.shape[1],X.shape[1]))).astype(int)

Principal component analysis is also closely related to eigen-decomposition and the singular value decomposition problem.
The former can be quickly verified that **pca.explained\_variance_** and **pca.components_** are, in fact, the **eigenvalues** and **eigenvectors** of the covariance matrix,
which also explains the orthogonality among all principal components.

$$ cov(X)Q = \Lambda Q $$

or for each column vector $\vec{q_j}$ in **Q**, this is the eigenequation:

$$ R \vec{q_j}=\lambda_j \vec{q_j}$$

where

$$R=cov(X) = E(X-\mu)(X-\mu)^T$$

In [ ]:
# covariance matrix of X
R = np.dot(X_centered.T, X_centered) / X.shape[0]

for eigenvalue, eigenvector in zip(pca.explained_variance_, pca.components_):
    print(np.allclose(
        np.dot(R, eigenvector),          # left hand side
        np.dot(eigenvalue, eigenvector)  # right hand side
    , rtol=1e-3))

## Scree plot

This plots variances (y-axis) against components (x-axis).
As one moves to the right, toward later components, the variances (or the eigenvalues) drop.
It helps us to decide on number of principal components to be retained, although this is subjective.

In [ ]:
x_ticks = np.arange(len(pca.components_))+1
plt.xticks(x_ticks) # this enforces integers on the x-axis
plt.plot(x_ticks, pca.explained_variance_)

Similarily, you could also plot the explained variance ratio.
A full PCA without components truncated would have total explained variance ratio equal to 1.

In [ ]:
plt.xticks(x_ticks)
plt.plot(x_ticks, pca.explained_variance_ratio_)
print('total expained variance ratio', np.sum(pca.explained_variance_ratio_))

The above two plots show that the first component explained most of the variance present in the data. We can simply work with first 2/3 compoents. 

## Conclusion

In this lab we learned about:
+ Apply PCA to a dataset for feature extraction.
+ Reconstruct an approximation of original dataset using principal components.
+ Eigenstructure of PCA.
+ Scree plot of PCA.